In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import tensorflow as tf
import numpy as np
from rotary_embedding_tensorflow import apply_rotary_emb, RotaryEmbedding
from fast_transformer import FastTransformer

In [3]:
from malaya.text.bpe import WordPieceTokenizer

In [4]:
tokenizer = WordPieceTokenizer('BERT.wordpiece', do_lower_case = False)
# tokenizer.tokenize('halo nama sayacomel')

In [5]:
import pickle

with open('relevancy-fastformer.pkl', 'rb') as fopen:
    train_input_ids, train_Y, test_input_ids, test_Y = pickle.load(fopen)
    
len(train_input_ids), len(train_Y), len(test_input_ids), len(test_Y)

(79940, 79940, 4228, 4228)

In [6]:
epoch = 3
batch_size = 8
warmup_proportion = 0.1
num_train_steps = int(len(train_input_ids) / batch_size * epoch)
num_warmup_steps = int(num_train_steps * warmup_proportion)

In [7]:
import optimization

In [8]:
def create_initializer(initializer_range=0.02):
    return tf.compat.v1.truncated_normal_initializer(stddev=initializer_range)

class Model:
    def __init__(
        self,
        dimension_output,
        learning_rate = 2e-5,
        training = True,
    ):
        self.X = tf.compat.v1.placeholder(tf.compat.v1.int32, [None, None])
        mask = tf.compat.v1.math.not_equal(self.X, 0)
        mask = tf.compat.v1.cast(mask, tf.compat.v1.bool)
        self.Y = tf.compat.v1.placeholder(tf.compat.v1.int32, [None])
        self.maxlen = tf.compat.v1.shape(self.X)[1]
        self.lengths = tf.compat.v1.count_nonzero(self.X, 1)
        
        self.model = FastTransformer(
            num_tokens = 32000,
            dim = 768,
            depth = 12,
            heads = 12,
            max_seq_len = 2048,
            absolute_pos_emb = True,
            mask = mask
        )
        self.logits = self.model(self.X)[0]
        self.logits_seq = tf.compat.v1.layers.dense(self.logits, dimension_output,
                                         kernel_initializer=create_initializer())
        
        self.logits_seq = tf.compat.v1.identity(self.logits_seq, name = 'logits_seq')
        self.logits = self.logits_seq[:, 0]
        self.logits = tf.compat.v1.identity(self.logits, name = 'logits')
        
        self.cost = tf.compat.v1.reduce_mean(
            tf.compat.v1.nn.sparse_softmax_cross_entropy_with_logits(
                logits = self.logits, labels = self.Y
            )
        )
        
        self.optimizer = optimization.create_optimizer(self.cost, learning_rate, 
                                                       num_train_steps, num_warmup_steps, False)
        correct_pred = tf.compat.v1.equal(
            tf.compat.v1.argmax(self.logits, 1, output_type = tf.compat.v1.int32), self.Y
        )
        self.accuracy = tf.compat.v1.reduce_mean(tf.compat.v1.cast(correct_pred, tf.compat.v1.float32))

In [11]:
dimension_output = 2
learning_rate = 2e-5

tf.compat.v1.reset_default_graph()
sess = tf.compat.v1.InteractiveSession()
model = Model(
    dimension_output,
    learning_rate
)

sess.run(tf.compat.v1.global_variables_initializer())
var_lists = tf.compat.v1.trainable_variables()

Tensor("fast_transformer/pre_norm/fast_attention/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm/fast_attention/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_2/fast_attention_1/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_2/fast_attention_1/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_4/fast_attention_2/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_4/fast_attention_2/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_6/fast_attention_3/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_6/fast_attention_3/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_8/fast_attention_4/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_8/fast_attention_4/Select:0", shape=(?, 12, ?), dtype=float32)
Tensor("fast_transformer/pre_norm_10/f

In [12]:
import collections
import re

def get_assignment_map_from_checkpoint(tvars, init_checkpoint):
    """Compute the union of the current variables and checkpoint variables."""
    assignment_map = {}
    initialized_variable_names = {}

    name_to_variable = collections.OrderedDict()
    for var in tvars:
        name = var.name
        m = re.match('^(.*):\\d+$', name)
        if m is not None:
            name = m.group(1)
        name_to_variable[name] = var

    init_vars = tf.compat.v1.train.list_variables(init_checkpoint)

    assignment_map = collections.OrderedDict()
    for x in init_vars:
        (name, var) = (x[0], x[1])
        if name not in name_to_variable:
            continue
        assignment_map[name] = name_to_variable[name]
        initialized_variable_names[name] = 1
        initialized_variable_names[name + ':0'] = 1

    return (assignment_map, initialized_variable_names)

In [13]:
tvars = tf.compat.v1.trainable_variables()
checkpoint = 'fastformer-base/model.ckpt-500000'
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, 
                                                                                checkpoint)

In [14]:
saver = tf.compat.v1.train.Saver(var_list = assignment_map)
saver.restore(sess, checkpoint)

INFO:tensorflow:Restoring parameters from fastformer-base/model.ckpt-500000


In [15]:
pad_sequences = tf.compat.v1.keras.preprocessing.sequence.pad_sequences

In [18]:
from tqdm import tqdm
import time

for EPOCH in range(1):

    train_acc, train_loss, test_acc, test_loss = [], [], [], []
    pbar = tqdm(
        range(0, len(train_input_ids), batch_size), desc = 'train minibatch loop'
    )
    for i in pbar:
        index = min(i + batch_size, len(train_input_ids))
        batch_x = train_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = train_Y[i: index]
        acc, cost, _ = sess.run(
            [model.accuracy, model.cost, model.optimizer],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        train_loss.append(cost)
        train_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    pbar = tqdm(range(0, len(test_input_ids), batch_size), desc = 'test minibatch loop')
    for i in pbar:
        index = min(i + batch_size, len(test_input_ids))
        batch_x = test_input_ids[i: index]
        batch_x = pad_sequences(batch_x, padding='post')
        batch_y = test_Y[i: index]
        acc, cost = sess.run(
            [model.accuracy, model.cost],
            feed_dict = {
                model.Y: batch_y,
                model.X: batch_x,
            },
        )
        test_loss.append(cost)
        test_acc.append(acc)
        pbar.set_postfix(cost = cost, accuracy = acc)
        
    train_loss = np.mean(train_loss)
    train_acc = np.mean(train_acc)
    test_loss = np.mean(test_loss)
    test_acc = np.mean(test_acc)
    
    print(
        'epoch: %d, training loss: %f, training acc: %f, valid loss: %f, valid acc: %f\n'
        % (EPOCH, train_loss, train_acc, test_loss, test_acc)
    )

train minibatch loop:  78%|███████▊  | 7842/9993 [1:09:02<17:53,  2.00it/s, accuracy=1, cost=0.00104]   IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

test minibatch loop: 100%|██████████| 529/529 [02:36<00:00,  3.38it/s, accuracy=1, cost=0.00193]   

epoch: 0, training loss: 0.301127, training acc: 0.911988, valid loss: 0.313241, valid acc: 0.920841



In [19]:
saver = tf.compat.v1.train.Saver(tf.compat.v1.trainable_variables())
saver.save(sess, 'fastformer-base-relevancy/model.ckpt')

'fastformer-base-relevancy/model.ckpt'

In [20]:
real_Y, predict_Y = [], []

pbar = tqdm(
    range(0, len(test_input_ids), batch_size), desc = 'validation minibatch loop'
)
for i in pbar:
    index = min(i + batch_size, len(test_input_ids))
    batch_x = test_input_ids[i: index]
    batch_x = pad_sequences(batch_x, padding='post')
    batch_y = test_Y[i: index]
    
    predict_Y += np.argmax(sess.run(model.logits,
            feed_dict = {
            model.X: batch_x,
            },
    ), 1, ).tolist()
    real_Y += batch_y

validation minibatch loop: 100%|██████████| 529/529 [01:46<00:00,  4.99it/s]


In [21]:
from sklearn import metrics

print(
    metrics.classification_report(
        real_Y, predict_Y, target_names = ['not relevant', 'relevant'],
        digits = 5
    )
)

              precision    recall  f1-score   support

not relevant    0.93553   0.86062   0.89651      1686
    relevant    0.91222   0.96066   0.93581      2542

    accuracy                        0.92077      4228
   macro avg    0.92387   0.91064   0.91616      4228
weighted avg    0.92151   0.92077   0.92014      4228



In [22]:
strings = ','.join(
    [
        n.name
        for n in tf.compat.v1.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'logits' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'ReadVariableOp' not in n.name
        and 'AssignVariableOp' not in n.name
        and '/Assign' not in n.name
        and '/Adam' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'dense/kernel',
 'dense/bias',
 'logits_seq',
 'logits']

In [23]:
def freeze_graph(model_dir, output_node_names):

    if not tf.compat.v1.io.gfile.exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.compat.v1.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.compat.v1.Session(graph = tf.compat.v1.Graph()) as sess:
        saver = tf.compat.v1.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.compat.v1.graph_util.convert_variables_to_constants(
            sess,
            tf.compat.v1.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.compat.v1.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [24]:
freeze_graph('fastformer-base-relevancy', strings)

INFO:tensorflow:Restoring parameters from fastformer-base-relevancy/model.ckpt
Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`
Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`
INFO:tensorflow:Froze 162 variables.
INFO:tensorflow:Converted 162 variables to const ops.
6727 ops in the final graph.


In [25]:
def load_graph(frozen_graph_filename):
    with tf.compat.v1.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.compat.v1.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.compat.v1.Graph().as_default() as graph:
        tf.compat.v1.import_graph_def(graph_def)
    return graph

In [26]:
g = load_graph('fastformer-base-relevancy/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
logits = g.get_tensor_by_name('import/logits_seq:0')

In [28]:
# %%time

# predicted = test_sess.run(logits,
#             feed_dict = {
#                 x: [parsed_sequence],
#             },
#     )[0]
# merged = merge_wordpiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
# print(list(zip(merged[0], merged[1])))

In [29]:
from tensorflow.tools.graph_transforms import TransformGraph

In [30]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

input_nodes = [
    'Placeholder',
]
output_nodes = [
    'logits',
    'logits_seq'
]

pb = 'fastformer-base-relevancy/frozen_model.pb'

input_graph_def = tf.compat.v1.GraphDef()
with tf.compat.v1.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                           input_nodes,
                                           output_nodes, transforms)
    
with tf.compat.v1.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

Instructions for updating:
Use tf.compat.v1.gfile.GFile.


In [31]:
# g = load_graph('fastformer-tiny-entities/frozen_model.pb.quantized')
# x = g.get_tensor_by_name('import/Placeholder:0')
# logits = g.get_tensor_by_name('import/logits:0')
# test_sess = tf.compat.v1.InteractiveSession(graph = g)

In [32]:
# %%time

# predicted = test_sess.run(logits,
#             feed_dict = {
#                 x: [parsed_sequence],
#             },
#     )[0]
# merged = merge_wordpiece_tokens_tagging(bert_sequence, [idx2tag[d] for d in predicted])
# print(list(zip(merged[0], merged[1])))

In [35]:
file = 'fastformer-base-relevancy/frozen_model.pb'
outPutname = 'relevancy/fastformer/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)

In [36]:
file = 'fastformer-base-relevancy/frozen_model.pb.quantized'
outPutname = 'relevancy/fastformer-quantized/model.pb'
b2_bucket.upload_local_file(
    local_file=file,
    file_name=outPutname,
    file_infos=file_info,
)